# Counterexample X

### Installations

The below will install the necessary packages to run the notebook.

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scipy

The following block imports the relevant modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.integrate import cumulative_trapezoid
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

The below helper functions will be useful later.

In [ ]:
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
import numpy as np

def compute_integral(x_vals, f_function, resolution=500, grid_threshold=50, normalise=True):
    """
    Compute F(x) = ∫₀ˣ f(t) dt for each x in x_vals.
    
    If len(x_vals) > grid_threshold, assume x_vals is already a proper grid
    and integrate directly using cumulative_trapezoid.

    Parameters
    ----------
    x_vals : array_like
        Array of x values (non-negative) at which to compute the integral.
    f_function : callable
        Function handle that accepts a 1D array and returns f(x).
    resolution : int
        Resolution for internal grid (used if x_vals is sparse).
    grid_threshold : int
        Number of points above which x_vals is treated as a proper grid.
    normalise : boolean
        Whether or not to normalise (divide by value at 1, used to account for computational innacuracies)

    Returns
    -------
    F_vals : ndarray
        Array of integral values F(x_vals).
    """
    x_vals = np.asarray(x_vals)
    assert np.all(x_vals >= 0), "x_vals must be non-negative"

    if len(x_vals) > grid_threshold:
        # Assume x_vals is already a suitable integration grid
        f_vals = f_function(x_vals)
        F_vals = cumulative_trapezoid(f_vals, x_vals, initial=0)
        
        # Normalise by making sure final entry is one
        if normalise:
            F_vals = F_vals/F_vals[-1]

        return F_vals

    # Otherwise: build a fine internal grid from 0 to 1
    x_grid = np.linspace(0, 1, resolution)
    f_grid = f_function(x_grid)

    F_grid = cumulative_trapezoid(f_grid, x_grid, initial=0)
    
    # Normalise by making sure final entry is one
    if normalise:
        F_grid = F_grid/F_grid[-1]

    # Interpolate cumulative integral back to sparse x_vals
    interp_func = interp1d(x_grid, F_grid, kind='nearest', bounds_error=False, fill_value="extrapolate")
    F_vals = interp_func(x_vals)
    
    return F_vals


In [ ]:
def plot_surface_3d(F_vals, x_vals, y_vals=None, downsample=1, title='Surface Plot', z_title='Z', sphere_radius=None):
    """
    Plots a 3D surface for F_vals over the given x and y values.

    Parameters:
    - F_vals: 2D array of values representing F(x, y)
    - x_vals: 1D array for x-axis (or 2d precomputed grid coordinates)
    - y_vals: 1D array for y-axis (or 2d precomputed grid coordinates) (optional; defaults to x_vals)
    - downsample: int factor to reduce resolution (e.g., 2 means every 2nd point)
    - sphere_radius: float, optional — if provided, a translucent sphere of this radius is added
    """

    if y_vals is None:
        y_vals = x_vals

    if y_vals.ndim != x_vals.ndim:
        raise ValueError('x and y must have same number of dimensions.')

    # Downsample data if needed
    if downsample > 1:
        F_vals = F_vals[::downsample, ::downsample]

        if x_vals.ndim == 1:
            x_vals = x_vals[::downsample]
            y_vals = y_vals[::downsample]
        else:
            x_vals = x_vals[::downsample, ::downsample]
            y_vals = y_vals[::downsample, ::downsample]

    # Create 2D grid (X, Y)
    if x_vals.ndim == 1:
        X, Y = np.meshgrid(x_vals, y_vals, indexing='xy')
    else:
        X = x_vals
        Y = y_vals

    # Create surface plot list
    surfaces = [
        go.Surface(
            x=X,
            y=Y,
            z=F_vals,
            colorscale='Viridis',
            name='Main Surface'
        )
    ]

    # Optionally add a translucent sphere
    if sphere_radius is not None:
        u = np.linspace(0, np.pi, 50)
        v = np.linspace(0, 2 * np.pi, 50)
        U, V = np.meshgrid(u, v)
        Xs = sphere_radius * np.sin(U) * np.cos(V)
        Ys = sphere_radius * np.sin(U) * np.sin(V)
        Zs = sphere_radius * np.cos(U)

        surfaces.append(
            go.Surface(
                x=Xs,
                y=Ys,
                z=Zs,
                opacity=0.3,
                showscale=False,
                colorscale='Blues',
                name=f'Sphere r={sphere_radius}'
            )
        )

    # Create figure
    fig = go.Figure(data=surfaces)

    # Layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title=z_title
        ),
        autosize=True,
        width=800,
        height=700
    )

    fig.show()


### Introduction

In this notebook, we shall construct a $C^1$ function $f:\mathbb{R}^3\rightarrow \mathbb{R}$ with the following properties:

 - $f(0,0,0)=0$.
 - The only critical point of $f$ occurs at the origin.
 - for every $\epsilon > 0$, the zero level set $f^{-1}(0)$ intersects the boundary of the ball $\partial B_\epsilon \subset \mathbb{R}^3$ tangentially.

To achieve this goal, we construct the function in two steps, with intermediate and final expressions given in spherical coordinates $(r, \theta, \psi)$, where $r \in [0,\infty)$, $\theta \in [0,\pi]$, and $\psi \in [0,2\pi)$:

- **Step 1:** Define a $C^1$ function $H : [0,\pi] \times [0, 2\pi) \rightarrow [0,\infty)$ such that the surface given by $r = H(\theta, \psi)$ is tangent to every ball of radius less than or equal to 1.

- **Step 2:** Construct a $C^1$ function $f : [0,\infty) \times [0,\pi] \times [0,2\pi) \rightarrow \mathbb{R}$ with zero level set given by
  
  $$f^{-1}(0) = \{ (r, \theta, \psi) \mid r = H(\theta, \psi) \},$$
  zero gradient at the origin, and non-vanishing gradient everywhere else within a neighbourhood of the origin.

We treat $[0,\infty) \times [0,\pi] \times [0,2\pi)$ as spherical coordinates on $\mathbb{R}^3$, with the usual identification of $\psi = 0$ and $\psi = 2\pi$, and the azimuthal degeneracy at the poles $\theta = 0$ and $\theta = \pi$.



### Step 1: Constructing $H$

To construct $H$, we begin in Cartesian coordinates $(x, y, z)$ by defining an intermediate function $H^\ast : \mathbb{R}^2 \to \mathbb{R}$ such that, for every $z \in [0,1]$, there exists a point $(x, y) \in [0,1]^2$ with $H^\ast(x, y) = z$ and $\nabla H^\ast(x, y) = 0$. This construction follows the approach of [Grinberg (2018)](link), but we provide explicit equations throughout.


To be specific, we construct $H$ in several steps:

1. **One-dimensional slice:** We begin by defining a function $\tilde{H} : [0,1] \to [0,1]$ that has a critical point at every level $c \in C$, where $C$ denotes the middle-thirds Cantor set.

2. **Symmetrization:** We extend this function to $[-1,1]$ by requiring it to be even.

3. **Product extension:** Identifying $[-1,1]$ with $[-1,1] \times \{0\}$, we extend the domain to $[-1,1]^2$ by defining  
   $$
   H^\ast(x, y) = \frac{1}{2}(\tilde{H}(x) + \tilde{H}(y)).
   $$

4. **Global extension:** Extending the definition of $H^\ast$ on $[-1,1]^2$ from above, we now define $H^\ast(x, y) = 1$ for all $(x, y) \in \mathbb{R}^2 \setminus [-2,2]^2$, and use a $C^1$ transition function to smoothly interpolate over the rectangular region $[-2,2]^2 \setminus [-1,1]^2$, resulting in a globally $C^1$ function.

5. **Mapping to Spherical:** We next define $H$ in terms of spherical coordinates $(\theta,\psi)$ as $H(\theta,\psi):=H^\ast(\frac{\pi}{2k}(\theta+k),\frac{\pi}{k}(\psi+k))$ for some suitably large $k$.

#### One Dimensional Slice

To construct the 1 dimensional function with critical points at every level $c \in C$, we shall use a fractal construction for the derivative of $\tilde{H}$ which involves carefully placing and summing spikes of precomputed areas.

We start by letting $n\in\mathbb{N}_{>0}$ and $k \in \{0, 1, \dots, 2^n - 1\}$, and write the binary expansion of $k$ as:

\begin{equation}\nonumber
k = \sum_{j=1}^n r_j \cdot 2^{n-j}, \quad \text{where } r_j \in \{0,1\}.
\end{equation}
We then define:
\begin{equation}\nonumber
a_{n,k} := \sum_{j=1}^n r_j \cdot \frac{3}{5} \cdot \left( \frac{2}{5} \right)^{j-1},\quad\text{and}\quad b_{n,k} := a_{n,k} + \left( \frac{2}{5} \right)^n.
\end{equation}


Next, we define the following 'spike' function for $x \in [0,1]$, $n \in \mathbb{N}_{>0}$, and $k \in \{0, \dots, 2^{n-1} - 1\}$:

$$
\text{Spike}_{k,n}(x) = 
\begin{cases}
    0 & \text{if } x \leq \frac{1}{2}(a_{n,k} + b_{n,k} - w_n), \\
    2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k}), \\
    h_n - 2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k}) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k}) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n), \\
    0 & \text{if } x \geq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n).
\end{cases}
$$

where the height and width are given by:

$$
h_n := 4 \left( \frac{5}{6} \right)^{n+1}, \qquad w_n := \frac{2}{h_n \cdot 3^{n+1}}.
$$

To give some intuition as to what this looks like, we provide a function below:

In [ ]:
import numpy as np

def spike(x, n, k):
    """
    Vectorized version of the spike function Spike_{k,n}(x) for NumPy arrays.
    
    Parameters:
        x : np.ndarray
            Input values in [0, 1]
        n : int
            Scale level (must be >= 1)
        k : int
            Index in {0, ..., 2^n - 1}
            
    Returns:
        np.ndarray : Values of the spike function at x
    """
    x = np.asarray(x)
    
    # Convert k to binary vector r = [r1, ..., rn]
    r = [(k >> (n - j - 1)) & 1 for j in range(n)]
    
    # Compute a_{n,k}
    a_nk = sum(r_j * (3/5) * (2/5)**j for j, r_j in enumerate(r))
    
    # Compute b_{n,k}
    b_nk = a_nk + (2/5)**n
    
    # Compute height and width
    h_n = 4 * (5/6)**(n + 1)
    w_n = 2 / (h_n * 3**(n + 1))
    
    # Midpoint and boundaries
    mid = 0.5 * (a_nk + b_nk)
    left = mid - 0.5 * w_n
    right = mid + 0.5 * w_n

    # Initialize result
    y = np.zeros_like(x)

    # Linear rising edge
    mask_rise = (x > left) & (x <= mid)
    y[mask_rise] = 2 * h_n / w_n * (x[mask_rise] - left)

    # Linear falling edge
    mask_fall = (x > mid) & (x < right)
    y[mask_fall] = h_n - 2 * h_n / w_n * (x[mask_fall] - mid)

    # Peak (only applies if a value matches mid exactly, which is rare)
    y[np.isclose(x, mid)] = h_n

    return y

And an example plot:

In [ ]:
import matplotlib.pyplot as plt

n = 0
k = 0
x_vals = np.linspace(0, 1, 1000)
y_vals = spike(x_vals, n, k)

plt.plot(x_vals, y_vals)
plt.title(f"Spike function for n={n}, k={k}")
plt.xlabel("x")
plt.ylabel(f"Spike_{{{k},{n}}}(x)")
plt.grid(True)
plt.show()

The 'spike' function creates a triangular signal, with carefully chosen width and position, which encloses an area of exactly $(\frac{2}{3})^{n+1}$.

The width and position of the spikes have been carefully chosen so that we may define a function $h$ as follows:

\begin{equation}\nonumber 
    h(x)= \sum_{n=0}^{\infty}\sum_{i=0}^{2^n-1} \text{Spike}_{k,n}(x).
\end{equation}

In [ ]:
def compute_h(x, N_max=5):
    """
    Compute an approximation of h(x) = sum_{n=0}^{∞} sum_{k=0}^{2^n - 1} Spike_{k,n}(x)
    using a finite sum up to N_max.
    
    Parameters:
        x : np.ndarray
            Array of input values in [0,1].
        N_max : int
            Maximum value of n to include in the approximation (n=0 to N_max).
    
    Returns:
        np.ndarray : Approximated h(x) values.
    """
    x = np.asarray(x)
    h_vals = np.zeros_like(x)

    for n in range(N_max + 1):
        for k in range(2**n):
            h_vals += spike(x, n, k)
    
    return h_vals

In [ ]:
x_vals = np.linspace(0, 1, 1000)
h_vals = compute_h(x_vals, N_max=6)

plt.plot(x_vals, h_vals)
plt.title("Approximation of h(x) with N_max=6")
plt.xlabel("x")
plt.ylabel("h(x)")
plt.grid(True)
plt.show()

The function $h$ is a fractal function consisting of infinitely many non-overlapping spikes. Due to the choices of $h_n$, $w_n$, $a_{n,k}$, and $b_{n,k}$ used in the construction, the spikes become shorter and narrower as $n$ increases. As a result, it is straightforward to verify that $h$ is continuous.

The construction begins with a single spike of area $\frac{2}{3}$, followed by 2 spikes of area $\left(\frac{2}{3}\right)^2$, then 4 spikes of area $\left(\frac{2}{3}\right)^3$, and so on. A simple calculation shows that the total area under the graph of $h$ over $[0,1]$ is exactly 1.

This reasoning can be extended further. As every $c \in C$ can be expressed as a base 3 expansion using only $0$s and $2$s, or equivalently as a linear combination of powers of $\frac{2}{3}$, it easily follows that, for all $c\in C$, there exists an $x \in [0,1]$ such that the cumulative area under $h$ from $0$ to $x$ is exactly $c$. This defines a bijection between $C$ and the set of zeros of $h$, where $h(x) = 0$ if and only if the cumulative area up to $x$ equals $c$.


Noting this this, we now define $\tilde{H}:[0,1]\rightarrow [0,1]$ as follows:

\begin{equation}\nonumber
    \tilde{H}(x):=\int_{t=0}^x h(t)dt.
\end{equation}


In [ ]:
def compute_H_tilde(x_vals, normalise=True):
    
    # Separate into negative and non-negative parts
    x_neg = x_vals[x_vals < 0]
    x_pos = x_vals[x_vals >= 0]

    # Prepare result array
    H_tilde_vals = np.empty_like(x_vals, dtype=float)

    # Compute H for negative values
    if len(x_neg) > 0:

        # Flip sign and reverse order
        x_neg_flipped = -x_neg[::-1]  

        # Compute integral for negative
        H_tilde_neg = compute_integral(x_neg_flipped, compute_h, normalise=normalise)
        
        # Reverse to match original x_neg order
        H_tilde_neg = H_tilde_neg[::-1]  

        # Insert back into array
        H_tilde_vals[:len(H_tilde_neg)] = H_tilde_neg

    # Compute H for positive values
    if len(x_pos) > 0:
        
        # Compute integral for positive
        H_tilde_pos = compute_integral(x_pos, compute_h, normalise=normalise)

        # Insert back into array
        H_tilde_vals[-len(H_tilde_pos):] = H_tilde_pos

    # Return results
    return H_tilde_vals

In [ ]:
x_vals = np.linspace(0, 1, 1000)
H_tilde_vals = compute_H_tilde(x_vals)

plt.plot(x_vals, H_tilde_vals)
plt.title(r"Approximation of $\tilde{H}$(x)")
plt.xlabel("x")
plt.ylabel(r"$\tilde{H}(x)$")
plt.grid(True)
plt.show()

#### Symmetrization

We then extend this definition to $[-1,1]$ by taking $\tilde{H}(x):=\tilde{H}(-x)$ for $x<0$. Note that, as $\tilde{H}'(0)=h(0)=0$, $\tilde{H}$ is continuously differentiable across the whole of $[-1,1]$.

In [ ]:
x_vals = np.linspace(-1, 1, 1000)
H_tilde_vals = compute_H_tilde(x_vals)

plt.plot(x_vals, H_tilde_vals)
plt.title(r"Approximation of $\tilde{H}$(x)")
plt.xlabel("x")
plt.ylabel(r"$\tilde{H}(x)$")
plt.grid(True)
plt.show()

By construction, $\tilde{H}$ has critical points at every level $c\in C$. 


#### Product Extension

Next, we define a two-dimensional function $H^\ast:[-1,1]^2\rightarrow [-1,1]$ by $H^\ast(x,y):=\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$.

In [ ]:
# Function to compute H
def compute_H_star(x_vals,y_vals=None,normalise=True):

    # Assume we are using the same axes for x and y if not given different
    if y_vals is None:
        y_vals = x_vals
        
    # Compute Hx and Hy
    Hx = compute_H_tilde(x_vals,normalise)      # shape (Nx,)
    Hy = compute_H_tilde(y_vals,normalise)      # shape (Ny,)

    # Use broadcasting to compute the outer sum
    H_star_vals = (Hx[:, np.newaxis] + Hy[np.newaxis, :]) / 2

    return(H_star_vals)

# Compute H_star_vals
H_star_vals = compute_H_star(x_vals)

In [ ]:
# Plot surface
plot_surface_3d(H_star_vals, x_vals, downsample=2, title="3D Surface With Critical Values [0,1]", z_title = "H*(x, y) = H\u0303(x) + H\u0303(y)")

Through exploiting a well known function of the Cantor set; $C+C=[0,2]$, it can be seen that $H$ has a critical value at every level $z\in[0,1]$.

#### Global Extension

Next, we extend $H^\ast$ to the entirety of $\mathbb{R}^2$. This is achieved using the $C^1$ transition function $s(x)$ defined by:

$$
s(x):=\begin{cases}
0 & x \leq 0, \\
\frac{x^2}{x^2+(1-x)^2} & 0 \leq x \leq 1, \\
1 & 1 \leq x. \\
\end{cases}
$$

We begin by extending $\tilde{H}$ to $\mathbb{R}$ by letting $\tilde{H}(x)=1$ for $x \not\in[-1,1]^2$. Next, we extend the definition of $H^\ast$ as follows:

$$H^\ast(x,y):=s\bigg(d\big((x,y),[-1,1]^2\big)\bigg)+\bigg(1-s\bigg(d\big((x,y),[-1,1]^2\big)\bigg)\bigg)\left(\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))\right),$$

where $d(p,A):=\inf_{a\in A}|p-a|$ represents the Euclidean distance from the point $p$ to the set $A$. 

The above notation is dense, but much easier to understand when visualised. When $d((x,y),[-1,1]^2)\geq 1$, we see that $H^\ast(x,y)=1$, and when $(x,y)\in [-1,1]^2$ we have that $d((x,y),[-1,1]^2)=0$ and thus $H^\ast(x,y)=\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$, as before. In the intermediate region, the function smoothly transitions from $\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$ to $1$.

In [ ]:
def extend_F(F_vals, c_vals, x_grid, y_grid, epsilon=1.0):
    """
    Constructs new_vals with correct epsilon-thickening logic:
      - new_vals = F_vals inside [-1, 1]^2
      - new_vals = T in thickened shell (distance <= epsilon, not box)
      - new_vals = k_vals outside the thickened region

    Parameters:
    - F_vals: np.ndarray, values of H(x, y)
    - c_vals: np.ndarray, values of k(x, y)
    - x_grid, y_grid: coordinate grids matching H_vals shape
    - epsilon: thickness parameter for blending

    Returns:
    - new_vals: np.ndarray, constructed values
    """

    # Due to transposed ordering induced by meshgrid, we need to transpose
    x_grid = x_grid.T
    y_grid = y_grid.T
    new_vals = np.zeros_like(F_vals)

    # Step 1: Compute distance from each point to [-1,1]^2 box
    dx = np.maximum(np.abs(x_grid) - 1, 0)
    dy = np.maximum(np.abs(y_grid) - 1, 0)
    dist = np.sqrt(dx**2 + dy**2)

    # Step 2: Define regions
    in_core = (np.abs(x_grid) <= 1) & (np.abs(y_grid) <= 1)
    in_thickened = (dist <= epsilon) & (~in_core)
    outside = dist > epsilon

    # Step 3: Compute blending weights in shell region
    d = dist
    denom = d**2 + (1 - d)**2
    denom[denom == 0] = 1  # Avoid division by zero
    w = d**2 / denom

    # Step 4: Assign values
    new_vals[in_core] = F_vals[in_core]
    new_vals[in_thickened] = w[in_thickened] * c_vals[in_thickened] + (1 - w[in_thickened]) * F_vals[in_thickened]
    new_vals[outside] = c_vals[outside]

    return new_vals

In [ ]:
# Global H star
# def compute_H_star_global(x_vals,y_vals):
    
#     # Compute H_star_vals
#     H_star_vals = compute_H_star(x_vals,y_vals)
    
#     # Constant values
#     c_vals = np.ones(H_star_vals.shape)
    
#     # Generate 2D grids from x_vals
#     X, Y = np.meshgrid(x_vals, y_vals, indexing='xy')

#     # Extend H*
#     H_star_vals = extend_F(H_star_vals, c_vals, X, Y, epsilon=1.0)
    
#     # Return result
#     return(H_star_vals)

# x values
x_vals = np.linspace(-6,6,1000)
y_vals = np.linspace(-6,6,1000)

# Extend H star
H_star_vals = compute_H_star_global(x_vals,y_vals)

# Plot result
plot_surface_3d(H_star_vals, x_vals, downsample=2, title="3D Surface With Critical Values [0,1]", z_title="H*(x, y) = H\u0303(x) + H\u0303(y)")

#### Mapping to Spherical

We next define $H$ in spherical coordinates $(\theta,\psi)$ by
$$
H(\theta,\psi) := H^*\left(\frac{\pi}{2k}(\theta + k),\, \frac{\pi}{k}(\psi + k)\right)
$$
for some suitably large integer $k$. Geometrically, this corresponds to embedding a small patch of the plane, containing the critical structure of $H^*$, onto the surface of a sphere of radius 1. Intuitively, one can think of this as removing a small 'panel' from the sphere and replacing it with a rescaled and smoothly curved version of $H^*$ that conforms to the spherical geometry. 

The below plot illustrates the surface $r=H(\theta,\psi)$, which by construction is tangent to every sphere with radius in $[0,1]$. MARKER add back in 2d spherical


In [ ]:
# Compute theta and psi
theta_vals = np.linspace(0,1,len(x_vals))*np.pi#(x_vals/2 + 1) * (np.pi/4)     # [π/4, 3π/4]
psi_vals = np.linspace(0,2,len(x_vals))*np.pi           # [π/2, 3π/2]

# Construct grid of (theta,psi) values
Theta, Psi = np.meshgrid(theta_vals, psi_vals, indexing='ij')  # shape (N, N)

# Compute R values on this grid
R = H_star_vals 

# Construct coordinates for visualisation
X = R * np.sin(Theta) * np.cos(Psi)
Y = R * np.sin(Theta) * np.sin(Psi)
Z = R * np.cos(Theta)

# Plot surface, sphere added to demonstrate tangency
plot_surface_3d(Z, X, Y, downsample=2,sphere_radius=None)

### Step 2: Constructing $f$

We now construct a function $f : [0,\infty) \times [0,\pi] \times [0,2\pi) \rightarrow \mathbb{R}$ whose zero level set corresponds to the surface defined by $H$, that is,
$$
f^{-1}(0) = \{ (r, \theta, \psi) \mid r = H(\theta, \psi) \},
$$
such that $f$ has vanishing gradient at the origin, and non-vanishing gradient in a neighborhood around the origin, excluding the origin itself.



To do so, we first define $S:\mathbb{R}\rightarrow [0,1]$ as $S(x):=s(|x|)$.

In [ ]:
def compute_S(x_values):
    x_abs = np.abs(x_values)
    S_values = np.zeros_like(x_abs)

    # Case 1: x_abs == 0
    S_values[x_abs == 0] = 0

    # CaseC 2: 0 < x_abs < 1
    mask_middle = (x_abs > 0) & (x_abs < 1)
    x_mid = x_abs[mask_middle]
    S_values[mask_middle] = x_mid**2 / (x_mid**2 + (1 - x_mid)**2)

    # Case 3: x_abs >= 1
    S_values[x_abs >= 1] = 1

    return S_values


In [ ]:
# Generate x values
x_values = np.linspace(-2, 2, 1000)
S_values = compute_S(x_values)

# Plotting
plt.plot(x_values, S_values, label='$S(x)$')
plt.title('Plot of $S(x) = s(|x|)$')
plt.xlabel('$x$')
plt.ylabel('$S(x)$')
plt.grid(True)
plt.legend()
plt.show()

Next, we define $f^\ast:\mathbb{R}^2 \times \mathbb{R}_{\geq 0} \rightarrow \mathbb{R}$ in cartesian coordinates as:

$$f^\ast(x,y,z):=-(S(x)+S(y))(z-H^\ast(x,y))z^2-\mathbb{1}[z> H^\ast(x,y)](z-H^\ast(x,y))^2,$$

where $\mathbb{1}[A]$ is the indicator function which evaluates to $1$ if the $A$ is true and $0$ otherwise.

In [ ]:
def compute_f_star(x_values, y_values, z_values):
    # Ensure numpy arrays (float64 by default)
    x_values = np.asarray(x_values)
    y_values = np.asarray(y_values)
    z_values = np.asarray(z_values)

    # Compute S(x) and S(y), shape for broadcasting
    Sx = compute_S((x_values+1)%2 -1)[:, np.newaxis, np.newaxis]     # (Nx, 1, 1)
    Sy = compute_S((y_values+1)%2 -1)[np.newaxis, :, np.newaxis]     # (1, Ny, 1)
    S_sum = Sx + Sy                                          # (Nx, Ny, 1)
    del Sx, Sy                                               # Free memory

    # Compute H*(x, y), shape for broadcasting with Z
    H_star = compute_H_star_global(x_values, y_values)       # (Nx, Ny)
    H_star = H_star[:, :, np.newaxis]                        # (Nx, Ny, 1)

    # Prepare Z broadcast (1, 1, Nz)
    Z = z_values[np.newaxis, np.newaxis, :]                  # (1, 1, Nz)

    # Compute Z - H*
    z_minus_H = Z - H_star                                   # (Nx, Ny, Nz)

    # Compute Z^2
    Z2 = Z * Z                                               # (1, 1, Nz)

    # Compute first term: S_sum * (Z - H*) * Z^2
    term1 = S_sum * z_minus_H
    del S_sum                                                # Free memory
    term1 *= Z2                                              # In-place multiplication

    # Compute second term: (Z - H*)^2 and mask it in-place
    z_minus_H_sq = z_minus_H * z_minus_H/4                   # (Nx, Ny, Nz)
    z_minus_H_sq[Z <= H_star] = 0                            # In-place masking

    # Final result
    f_star = -20*term1 - z_minus_H_sq
    del term1, z_minus_H_sq                                  # Cleanup

    return f_star                                            # Shape: (Nx, Ny, Nz)


The below creates several 3D plots of different slices of $f$.

In [ ]:
def plot_slice_of_f_star(
    axis='z',
    slice_value=0.0,
    x_range=np.linspace(-4, 4, 500),
    y_range=np.linspace(-4, 4, 500),
    z_range=np.linspace(0, 1.5, 500)
):
    assert axis in {'x', 'y', 'z'}, "axis must be 'x', 'y', or 'z'"

    if axis == 'z':
        f_vals = compute_f_star(x_range, y_range, np.array([slice_value]))
        f_slice = f_vals[:, :, 0]
        X, Y = np.meshgrid(x_range, y_range, indexing='ij')
        xlabel, ylabel = 'x', 'y'

    elif axis == 'y':
        f_vals = compute_f_star(x_range, np.array([slice_value]), z_range)
        f_slice = f_vals[:, 0, :]
        X, Y = np.meshgrid(x_range, z_range, indexing='ij')
        xlabel, ylabel = 'x', 'z'

    elif axis == 'x':
        f_vals = compute_f_star(np.array([slice_value]), y_range, z_range)
        f_slice = f_vals[0, :, :]
        X, Y = np.meshgrid(y_range, z_range, indexing='ij')
        xlabel, ylabel = 'y', 'z'
    
    # Ensure all arrays are float64 and same shape
    X = X.astype(float)
    Y = Y.astype(float)
    Z = f_slice.astype(float)

    if not (X.shape == Y.shape == Z.shape):
        raise ValueError(f"Shape mismatch: X{X.shape}, Y{Y.shape}, Z{Z.shape}")

    # Replace any NaNs or infs just in case
    Z = np.nan_to_num(Z, nan=0.0, posinf=0.0, neginf=0.0)

    # Plot
    fig = go.Figure(data=[
        go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')
    ])

    fig.update_layout(
        title=f"Slice of f*(x, y, z) at {axis} = {slice_value}",
        scene=dict(
            xaxis_title=xlabel,
            yaxis_title=ylabel,
            zaxis_title='f*',
        ),
        autosize=True,
        margin=dict(l=0, r=0, b=0, t=40)
    )

    
    fig.add_trace(go.Surface(
    x=X, y=Y, z=Z,
    colorscale='Viridis',
    showscale=True,
    contours=dict(
        z=dict(show=True, start=0, end=0.01, size=1e-1, color='red', width=6)
    ),
    opacity=0.95
    ))
    
    fig.show()


In [ ]:
plot_slice_of_f_star(axis='y', slice_value=0.0)

In [ ]:
plot_slice_of_f_star(axis='y', slice_value=0.5)

In [ ]:
plot_slice_of_f_star(axis='y', slice_value=-0.5)

In [ ]:
plot_slice_of_f_star(axis='z', slice_value=0.6)

Note that as $f^\ast$ is a composition of $C^1$ functions, it is $C^1$. We shall now show the following properties of $f^\ast$.

 1. The preimage under $f^\ast$ of $0$ is
    $$(f^\ast)^{-1}(0)=\{(x,y,z):z=H^\ast(x,y)\}\cup \{(x,y,z):z=0\}.$$
 2. For all $x,y\in\mathbb{R}$, the gradient of $f^\ast$ at $(x,y,0)$ is
    $$\nabla f^\ast(x,y,0)=0.$$
 3. For all $x,y\in\mathbb{R}$ and $z\in(0,\frac{2}{3})$, the gradient of $f^\ast$ at $(x,y,z)$ is nonzero.

**Proof of Property 1:** For the first property consider

**Proof of Property 2:**

**Proof of Property 3:** Assume $x,y\in \mathbb{R}_{\geq 0}$ and $z\in (0,1)$. Differentiating $f^\ast$ with respect to $z$ yeilds:

$$\frac{\partial f^\ast}{\partial z}(x,y,z)=-(S(x)+S(y))(3z-2H^\ast(x,y))z-\mathbb{1}[z> H^\ast(x,y)]2(z-H^\ast(x,y))$$

Suppose first that $z > H^\ast(x,y)$. Then we have that $3z-2H^\ast(x,y)\geq H^\ast(x,y) \geq 0$ and $2(z-H^\ast(x,y))>0$. As $S(x)+S(y)\geq 0$ by construction, it follows that the above partial derivative is the sum of a non-positive term and a strictly negative term. It follows that it is strictly negative, and thus non-zero.

Now, suppose that $z \leq H^\ast(x,y)$. Then the above is equal to zero if and only if one of the following conditions hold:
 - $S(x)+S(y)=0$
 - $3z-2H^\ast(x,y)=0$
 - $z=0$

In the first case, note that $S(x) = 0$ if and only if $x=0$ and is positive otherwise. Thus, we necessarily have that $x=y=0$. However, this implies $H^\ast(x,y)=0<z$, which contradicts the assumption that $z \leq H^\ast(x,y)$. Therefore, the first case cannot happen. Similarly, the third case is ruled out by the fact that $z\in (0,1)$. Thus, we need only consider the second.

Now, consider the partial derivative with respect to $x$.

$$\frac{\partial f^\ast}{\partial x}(x,y,z)=-S'(x)(z-H^\ast(x,y))z^2+\frac{1}{2}(S(x)+S(y))\tilde{H}'(x)z^2+\mathbb{1}[z>H^\ast(x,y)]H^\ast(x,y)\tilde{H}(x)$$

Evaluating this at $z=\frac{2}{3}H^\ast(x,y)$ gives the following:

$$\frac{\partial f^\ast}{\partial x}\bigg(x,y,\frac{2}{3}H^\ast(x,y)\bigg)=\frac{4}{27}S'(x)H^\ast(x,y)^3+\frac{4}{18}(S(x)+S(y))\tilde{H}'(x)H^\ast(x,y)^2$$

$$=\frac{4}{9}H^\ast(x,y)^2\bigg(\frac{1}{3}S'(x)H^\ast(x,y)+\frac{1}{2}(S(x)+S(y))\tilde{H}'(x)\bigg).$$

Note that, by assuming $z>0$, so $H^\ast(x,y)>0$. Suppose $x\in(0,1)$. Then $S'(x)>0$, $S(x)+S(y)>0$ and $\tilde{H}'(x)>0$. Thus, we have that the above is positive. By identical reasoning, we have that if $y \in (0,1)$ then:
$$\frac{\partial f^\ast}{\partial y}\bigg(x,y,\frac{2}{3}H^\ast(x,y)\bigg)>0.$$

Combining the above we see that the only way for the partial derivatives with respect to $x,y$ and $z$ to be simultaneously zero is if:

$$x>1, y> 1, z=\frac{2}{3}H^\ast(x,y).$$

(Note that the case $x=y=0$ is ruled out as this would imply $z=\frac{2}{3}H^\ast(x,y)=0$, which would contradict the assumption that $z>0$.) The above conditions, when taken together imply that $z\geq \frac{2}{3}$. Finally, we note that we can extend this argument to $x,y\in \mathbb{R}$ by noting that $f^\ast$ is even in $x$ and $y$. It now follows that $\nabla f^\ast(x,y,z) \neq 0$ for $z\in(0,\frac{2}{3})$.

In [ ]:
# Coordinates in spherical space
theta_vals = np.linspace(0,np.pi)
psi_vals = np.linspace(0,2*np.pi)
R_vals = np.linspace(0,1.5,100)

# Convert appropriately
theta_vals_converted = k*2*theta_vals/np.pi
psi_vals_converted = k*psi_vals/np.pi
R_vals_converted = np.array(R_vals)

# Compute f_vals
f_vals = compute_f_star(theta_vals_converted, psi_vals_converted, R_vals_converted)